In [38]:
####################################################################################################
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names) # organizando em um df
df['target'] = iris.target # rotulos

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)
modelo = LogisticRegression()
modelo.fit(X_train, y_train) # treinando o modelo

w = modelo.coef_[0] # pesos do modelo treinado
Vs = X_test.iloc[0].to_dict() # utilizxando a primeira instância dos dados de teste 

delta = []

# Calcula o valor delta para cada feature
# peso negativo, valor menor = maior possibilidade da instância peetencer a classe
# delta c\ peso negativo é (valor feature na instancia - valor min da feature no dataset) * o peso da feature 
# delta c\ peso positivo é (vvalor max da feature no dataset - valor feature na instancia - ) * o peso da feature
for i, feature in enumerate(df.columns[:-1]):  
    if w[i] < 0:
        delta.append((Vs[feature] - df[feature].min()) * w[i])
    else:
        delta.append((df[feature].max() - Vs[feature]) * w[i])

# Calcula o limiar (-Gamma_w)
R = abs(-sum(delta))  # Usando a equação 13 do artigo e garantindo que R seja positivo
# R = O valor absoluto da soma de todos os deltas

def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    # Ordena o delta junto com seus índices, em ordem decrescente de valor absoluto
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True) # enumerando os valores de delta em tuplas e ordenanas
                                                                                   # func lambda recebe a tupla produzida com o valor absoluto em ordem decrescente, do maior para o menor.
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    # Limite para considerar uma feature como importante, com base no limar R
    threshold_delta = 0.15 * R # Valor de delta mínimo para ser considerado relevante
    
    while R_atual > 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Desempacota o índice(soted_idx) e o valor(delta_value) do delta de acordo com o indce(idx)
        feature = X_test.columns[sorted_idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação apenas se o delta for maior que o threshold que está sendo calculado com uma % do R. não sei se esta certo isso
        if abs(delta_value) > threshold_delta:  # Verifica se o delta tem impacto relevante 
            Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice também para o loop
    
    return Xpl  # Retorna a PI-explicação

# Computa a PI-explicação
Xpl = one_explanation(Vs, delta, R)

#print(f"Pesos: {w}")
#print(f"Delta: {delta}")
#print(f"Limiar R: {R:.2f}")
#print(f"Vs: {Vs}\n")
print(f"PI-Explicação: ")
for item in Xpl:
    print(f"- {item}")

Pesos: [-0.39381403  0.96220565 -2.37519275 -0.99872731]
Delta: [-0.7088652578943643, 1.539529034488898, -8.788213178604362, -1.098600036798368]
Limiar R: 9.06
Vs: {'sepal length (cm)': 6.1, 'sepal width (cm)': 2.8, 'petal length (cm)': 4.7, 'petal width (cm)': 1.2}

PI-Explicação: 
- petal length (cm) - 4.7
- sepal width (cm) - 2.8
